# Run all 

In [4]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data
from src.scrapers.abstract_scraper import AbstractScraper
from src.scrapers.afry_scraper import AfryScraper
from src.scrapers.aliant_scraper import AliantScraper
from src.scrapers.asociety_scraper import ASocietyScraper
from src.scrapers.cinode_scraper import CinodeScraper
from src.scrapers.combitech_scraper import CombitechScraper
from src.scrapers.emagine_scraper import EmagineScraper
from src.scrapers.ework_scraper import EworkScraper
from src.scrapers.itcnetwork_scraper import ITCNetworkScraper
from src.scrapers.nikita_scraper import NikitaScraper
from src.scrapers.profinder_scraper import ProfinderScraper
from src.scrapers.regent_scraper import RegentScraper
from src.scrapers.senterprise_scraper import SenterpriseScraper
from src.scrapers.sigma_scraper import SigmaScraper
from src.scrapers.tingent_scraper import TingentScraper
from src.scrapers.upgraded_scraper import UpgradedScraper


scrapers = [AfryScraper(), AliantScraper(), ASocietyScraper(), CinodeScraper(), CombitechScraper(), EmagineScraper(), EworkScraper(), ITCNetworkScraper(), NikitaScraper(), ProfinderScraper(), RegentScraper(), SenterpriseScraper(), SigmaScraper(), TingentScraper(), UpgradedScraper()]
scrapers = [SenterpriseScraper()] #Cindoe

#scrapers = []

nr_ads_pre = len(load_local_data())

for s in scrapers:
    response = s.request_status()
    scraped_payload_dict = s.scrape_jobs_payloads_dict(response)
    
    old_payload_data = load_local_data(file_name='jobs_payloads.csv')
    old_payload_dict = dict(zip(old_payload_data['id'], old_payload_data['raw_payload']))
    old_bronze_data = load_local_data()
    
    new_payload_dict = s.return_new_payloads(new_dict=scraped_payload_dict, old_dict=old_payload_dict)
    new_bronze_data = s.parse_bronze_data(new_payload_dict)

    updated_payload_dict = s.concat_dicts(new_payload_dict, old_payload_dict)
    updated_payload_data = pd.DataFrame(list(updated_payload_dict.items()), columns=['id','raw_payload'])
    updated_bronze_data = s.concat_new_rows(new_bronze_data, old_bronze_data)

    #unload_local_data(updated_payload_data, file_name='jobs_payloads.csv')
    #unload_local_data(updated_bronze_data)

nr_ads_post = len(pd.read_csv('../data/bronze/jobs.csv'))
print('Total number of new added jobs:', nr_ads_post-nr_ads_pre)
updated_bronze_data.loc[updated_bronze_data['site'] ==s.__class__.site]



Senterprise > Response: 200
Senterprise > Nmr of scraped adds: 8
Senterprise > Parsing bronze data: 8
Total number of new added jobs: 0


,id,site,site_id,job_title,area,due_date,work_location,work_type,link,ingestion_ts,is_new
806,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6504216-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6504216-konsu...,2025-09-25 14:29:17,True
807,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6487493-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6487493-konsu...,2025-09-25 14:29:17,True
808,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6484496-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6484496-konsu...,2025-09-25 14:29:17,True
809,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6475370-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6475370-konsu...,2025-09-25 14:29:17,True
810,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6464556-konsu...,None,None,None,Stockholm,None,https://jobb.senterprise.se/jobs/6464556-konsu...,2025-09-25 14:29:17,True
811,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6438444-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6438444-konsu...,2025-09-25 14:29:17,True
812,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6438010-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6438010-konsu...,2025-09-25 14:29:17,True
813,Senterprise-https://jobb.senterprise.se/jobs/6...,Senterprise,https://jobb.senterprise.se/jobs/6421292-konsu...,None,None,None,Stockholm,Hybridarbete,https://jobb.senterprise.se/jobs/6421292-konsu...,2025-09-25 14:29:17,True


# Version 2

In [ ]:
import sys, os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from src.data_loader import load_local_data, unload_local_data
from src.scrapers.abstract_scraper import AbstractScraper
from src.scrapers.afry_scraper import AfryScraper
from src.scrapers.aliant_scraper import AliantScraper
from src.scrapers.asociety_scraper import ASocietyScraper
from src.scrapers.cinode_scraper import CinodeScraper
from src.scrapers.combitech_scraper import CombitechScraper
from src.scrapers.emagine_scraper import EmagineScraper
from src.scrapers.ework_scraper import EworkScraper
from src.scrapers.itcnetwork_scraper import ITCNetworkScraper
from src.scrapers.nikita_scraper import NikitaScraper
from src.scrapers.profinder_scraper import ProfinderScraper
from src.scrapers.regent_scraper import RegentScraper
from src.scrapers.senterprise_scraper import SenterpriseScraper
from src.scrapers.sigma_scraper import SigmaScraper
from src.scrapers.tingent_scraper import TingentScraper
from src.scrapers.upgraded_scraper import UpgradedScraper


scrapers = [AfryScraper(), AliantScraper(), ASocietyScraper(), CinodeScraper(), CombitechScraper(), EmagineScraper(), EworkScraper(), ITCNetworkScraper(), NikitaScraper(), ProfinderScraper(), RegentScraper(), SenterpriseScraper(), SigmaScraper(), TingentScraper(), UpgradedScraper()]
scrapers = [SenterpriseScraper()] #Cindoe
#scrapers = []

nr_ads_pre = len(load_local_data())

for s in scrapers:
    response = s.request_status()
    job_payloads = s.extract_job_payloads(response)
    scraped_data = s.scrape_all_jobs(job_payloads)
    
    old_payload_data = load_local_data(file_name='jobs_payloads.csv')
    old_bronze_data = load_local_data()
    old_data = old_bronze_data.merge(old_payload_data, on="id")
    print('old data:', len(old_data))
    
    new_scraped_data = s.return_new_rows(scraped_data, old_bronze_data)
    print('new data:', len(new_scraped_data))

    updated_data = AbstractScraper.set_dtypes(pd.concat([old_data, new_scraped_data], ignore_index=True))
    print('updated data:', len(updated_data))
    
    updated_bronze_data = updated_data[AbstractScraper.bronze_columns]
    updated_payload_data = updated_data[AbstractScraper.payload_columns]
    #unload_local_data(updated_payload_data, file_name='jobs_payloads.csv')
    #unload_local_data(updated_bronze_data)

nr_ads_post = len(pd.read_csv('../data/bronze/jobs.csv'))
print('Total number of new added jobs:', nr_ads_post-nr_ads_pre)
updated_data.loc[updated_data['site'] ==s.site]

Senterprise > Response: 200
Senterprise > Nmr of scraped adds: 8
old data: 806
new data: 8
updated data: 814
Total number of new added jobs: 0


,id,site,site_id,job_title,area,due_date,work_location,work_type,link,ingestion_ts,is_new,raw_payload
806,Senterprise-None,Senterprise,None,None,None,None,Stockholm,Hybridarbete,None,2025-09-25 14:26:57,True,"<li class=""w-full"">\n<div class=""relative flex..."
807,Senterprise-None,Senterprise,None,None,None,None,Stockholm,Hybridarbete,None,2025-09-25 14:26:57,True,"<li class=""w-full"">\n<div class=""relative flex..."
808,Senterprise-None,Senterprise,None,None,None,None,Stockholm,Hybridarbete,None,2025-09-25 14:26:57,True,"<li class=""w-full"">\n<div class=""relative flex..."
809,Senterprise-None,Senterprise,None,None,None,None,Stockholm,Hybridarbete,None,2025-09-25 14:26:57,True,"<li class=""w-full"">\n<div class=""relative flex..."
810,Senterprise-None,Senterprise,None,None,None,None,Stockholm,None,None,2025-09-25 14:26:57,True,"<li class=""w-full"">\n<div class=""relative flex..."
811,Senterprise-None,Senterprise,None,None,None,None,Stockholm,Hybridarbete,None,2025-09-25 14:26:57,True,"<li class=""w-full"">\n<div class=""relative flex..."
812,Senterprise-None,Senterprise,None,None,None,None,Stockholm,Hybridarbete,None,2025-09-25 14:26:57,True,"<li class=""w-full"">\n<div class=""relative flex..."
813,Senterprise-None,Senterprise,None,None,None,None,Stockholm,Hybridarbete,None,2025-09-25 14:26:57,True,"<li class=""w-full"">\n<div class=""relative flex..."


In [10]:
payload = job_payloads[0]
tag_link = payload.find("a",  href=True)
link = tag_link['href']
print(link)
print(payload)
print(tag_link)
link = s.extract_link(job_payloads[0])
print(link)


https://jobb.senterprise.se/jobs/6504216-konsultuppdrag-dataops-devops-specialist-gcp-migration
<li class="w-full">
<div class="relative flex flex-col items-center py-6 text-center sm:px-6 lg:px-16 hover:bg-gradient-block-base-bg">
<a class="w-fit max-w-full sm:min-w-[25%] sm:truncate hyphens-auto" data-turbo="false" href="https://jobb.senterprise.se/jobs/6504216-konsultuppdrag-dataops-devops-specialist-gcp-migration">
<span class="absolute inset-0"></span>
      Konsultuppdrag | DataOps / DevOps Specialist – GCP Migration
</a>
<div class="mt-1 text-md">
<span>Konsultuppdrag</span>
<span class="mx-[2px]">·</span>
<span>Stockholm</span>
<span class="mx-[2px]">·</span>
<span class="inline-flex items-center gap-x-2">
    Hybridarbete
    <i class="w-4 h-5 text-xs fas fa-wifi"></i>
</span>
</div>
</div>
<span class="block w-full h-px bg-gradient-block-base-border"></span>
</li>
<a class="w-fit max-w-full sm:min-w-[25%] sm:truncate hyphens-auto" data-turbo="false" href="https://jobb.senterp

In [ ]:
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup

import re
import requests
from bs4 import BeautifulSoup
import re
import requests


class ProfinderScraper:

    def clean_title(self, title: str):
        """Remove 'ID:xxx' from title."""
        return re.sub(r"\s*ID:\d+", "", title).strip()

    def extract_id(self, title: str):
        """Extract job ID if present."""
        match = re.search(r"ID:(\d+)", title)
        return match.group(1) if match else None

    def extract_location(self, title: str):
        """Try to extract location if mentioned with 'i <Ort>'."""
        match = re.search(r" i ([A-ZÅÄÖa-zåäö\s-]+)", title)
        return match.group(1).strip() if match else None


    def scrape_all_jobs(self, max_pages: int = 5):
        all_jobs = []
        for page in range(1, max_pages + 1):
            data = self.get_page(page)
            posts = data.get("postFeedPage", {}).get("posts", {}).get("posts", [])
            if not posts:
                break
            for post in posts:
                title = post.get("title", "")
                job = {
                    "id": self.extract_id(title),
                    "title": self.clean_title(title),
                    "location": self.extract_location(title),
                    "excerpt": post.get("excerpt"),
                    "url": post.get("link"),
                    "published": post.get("firstPublishedDate"),
                }
                all_jobs.append(job)
            print(f"{self.__class__.site} > Scraped {len(posts)} jobs from page {page}")
        return all_jobs





## Load bronze table 

In [ ]:
import pandas as pd
from src.data_loader import load_local_data, unload_local_data
bronze_data = load_local_data()
df = bronze_data.copy()
print(len(df['work_location'].unique()))
df['work_location'].unique()
display(df)
